In [ ]:
from shared.preface import *
import shared.functions as fct


def cell_gravity_short_range(
    cell_coords, cell_gen, init_GRID_S,
    DM_pos, DM_lim, DM_sim_mass, smooth_l,
    out_dir, fname
):
    # Center all DM positions w.r.t. cell center.
    DM_pos -= cell_coords

    # Cell lengths to limit DM particles. Limit for the largest cell is 
    # GRID_S/2, not just GRID_S, therefore the cell_gen+1 !
    cell_len = np.expand_dims(init_GRID_S/(2**(cell_gen+1)), axis=1)

    # Select DM particles inside each cell based on cube length generation.
    DM_in_cell_IDs = np.asarray(
        (np.abs(DM_pos[:,:,0]) < cell_len) & 
        (np.abs(DM_pos[:,:,1]) < cell_len) & 
        (np.abs(DM_pos[:,:,2]) < cell_len)
    )

    # Set DM outside cell to nan values.
    DM_pos[~DM_in_cell_IDs] = np.nan

    # Sort all nan values to the bottom of axis 1, i.e. the DM-in-cell-X axis 
    # and truncate array based on DM_lim parameter. This simple way works since 
    # each cell cannot have more than DM_lim.

    ind_2D = DM_pos[:,:,0].argsort(axis=1)
    ind_3D = np.repeat(np.expand_dims(ind_2D, axis=2), 3, axis=2)
    DM_sort = np.take_along_axis(DM_pos, ind_3D, axis=1)
    DM_in = DM_sort[:,:DM_lim,:]
    del ind_2D, ind_3D, DM_sort

    # Calculate distances of DM and adjust array dimensionally.
    DM_dis = np.expand_dims(np.sqrt(np.sum(DM_in**2, axis=2)), axis=2)

    # ------------------------------ #
    # Calculate short-range gravity. #
    # ------------------------------ #

    # Offset DM positions by smoothening length of Camila's simulations.
    eps = smooth_l / 2.

    # nan values to 0 for numerator, and 1 for denominator to avoid infinities.
    quot = np.nan_to_num(cell_coords - DM_in, copy=False, nan=0.0) / \
        np.nan_to_num(
            np.power((DM_dis**2 + eps**2), 3./2.), copy=False, nan=1.0
        )
    del DM_in, DM_dis
    derivative= G*DM_sim_mass*np.sum(quot, axis=1)
    del quot
    
    # note: Minus sign, s.t. velocity changes correctly (see GoodNotes).
    dPsi_short = np.asarray(-derivative, dtype=np.float64)
    np.save(f'{out_dir}/dPsi_grid_{fname}_short_range.npy', dPsi_short)


def cell_gravity_long_range(
    cellX_coords, cell_com, 
    DM_count, DM_sim_mass, smooth_l,
    out_dir, fname,
):
    
    # Each core gets 1 cell (cellX_coords). Then one less layer of broadcasting 
    # than in current function is needed.

    # Distances between cell centers and cell c.o.m. coords.
    com_dis = np.sqrt(np.sum((cellX_coords-cell_com)**2, axis=2))

    # Offset DM positions by smoothening length of Camila's simulations.
    eps = smooth_l / 2.

    # Long-range gravity component for each cell (without including itself).
    quot = (cellX_coords-cell_com)/np.power((com_dis**2 + eps**2), 3./2.)
    derivative = G*DM_sim_mass*np.sum(DM_count*quot, axis=...) #! Check axis...
    del quot_long

    # note: Minus sign, s.t. velocity changes correctly (see GoodNotes).
    dPsi_long = np.asarray(-derivative, dtype=np.float64)
    np.save(f'{out_dir}/dPsi_grid_{fname}_long_range.npy', dPsi_long)